In [2]:
# !apt-get install --no-install-recommends ffmpeg && pip install ffmpeg scikit-video
import skvideo.io
import skvideo.datasets
import imageio

datapath = "_data/"

# Instagram Event Highlighter
---
> ### Team4 | Taeyong Song, Jungin Park, Wonil Song and Eungbean Lee

All codes : https://github.com/eungbean/Computer-Vision-Cource-Project



In [3]:
#write a hashtag you desire
hashtag = "giriboyconcert"

## 0. Conventinal System Results

In [6]:
from IPython.display import Video
Video("_data/result-before.mp4", width=480, height=360) 

## 1. Instagram Scrapper  
credit: https://github.com/rarcega/instagram-scraper

In [19]:
# !instagram-scraper demo_ig_scraper --tag  
!cd _data && ls
Video("_assets/stack_original.mp4", width=480, height=360)

401.mp4  404.mp4  407.mp4  410.mp4  413.mp4	     result-before.mp4
402.mp4  405.mp4  408.mp4  411.mp4  414.mp4
403.mp4  406.mp4  409.mp4  412.mp4  alignment.ipynb


## 2. Clip Alignment  
credit: https://github.com/jeorgen/align-videos-by-sound


<img src="_assets/spectrogram1.png" width="400px" height="200px">
<img src="_assets/spectrogram2.png" width="400px" height="200px">

![pipeline](_assets/videosync.png)

In [8]:
alignment_result = !alignment_info_by_sound_track _data/*.mp4
alignment_result

["1567136358.987116| INFO:align#summarize_audiotrack:for '401.mp4' begin",
 "1567136358.987679| INFO:align#summarize_audiotrack:extracting audio tracks for '401.mp4' begin",
 "1567136359.073051| INFO:align#summarize_audiotrack:extracting audio tracks for '401.mp4' end",
 "1567136370.679762| INFO:align#summarize_audiotrack:for '401.mp4' end",
 "1567136370.679840| INFO:align#summarize_audiotrack:for '402.mp4' begin",
 "1567136370.679972| INFO:align#summarize_audiotrack:extracting audio tracks for '402.mp4' begin",
 "1567136370.759999| INFO:align#summarize_audiotrack:extracting audio tracks for '402.mp4' end",
 "1567136378.620474| INFO:align#summarize_audiotrack:for '402.mp4' end",
 "1567136378.620611| INFO:align#summarize_audiotrack:for '403.mp4' begin",
 "1567136378.620754| INFO:align#summarize_audiotrack:extracting audio tracks for '403.mp4' begin",
 "1567136378.725075| INFO:align#summarize_audiotrack:extracting audio tracks for '403.mp4' end",
 "1567136390.413738| INFO:align#summarize

### Regex
<img src="http://kevinvo.xyz/images/regex.png" width="600px">

Make the tedius outputs into desired list.

In [9]:
import re

text = alignment_result.s
delays = re.findall("\d\d\.\d\d\d(?=\')", text)

#str to float
delays = list(map(float, delays))
delays

[0.0,
 5.248,
 4.757,
 5.739,
 8.704,
 9.813,
 11.563,
 8.277,
 12.288,
 17.557,
 21.739,
 25.707,
 30.165,
 26.923,
 30.251]

## 3. Audio Stitch

Merge the audio pieces into make full-size audio.

In [11]:
from pydub import AudioSegment
import IPython.display as ipd
#load audio files from mp4
audios=[]
for index_video in range(401,400+len(delays)):
    audios.append(AudioSegment.from_file(datapath + str(index_video) + ".mp4", "mp4"))

In [12]:
audio_before = audios[0]#.fade_out(3000)
audio_middle = audios[9][42.652*1000:]#.fade_in(2000)
audio_after = audios[13][1.7*1000:]#.fade_in(2000)

audio_concat = audio_before + audio_middle + audio_after
audio_concat.export("_assets/mashup.mp3", format="mp3") #Save as mp3

ipd.Audio("_assets/mashup.mp3") #Load saved mp3 file

### Zoom in to stitch region

In [13]:
audio_concat[58*1000:62*1000:].export("_assets/mashup_zoomin.mp3", format="mp3") #Save as mp3
ipd.Audio("_assets/mashup_zoomin.mp3") #Load saved mp3 file

## 4. Zero Padding

<img src="_assets/video_padded.png">

In [ ]:
# !pip install moviepy
from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_merge_video_audio

outputpath = "_output/"
#LOAD MASTER AUDIO
audio_master = AudioFileClip("_assets/Audio_Full.mp3")

#DEFINE BLACK MASK
blackmask = ColorClip(size=(480,480), color=[0,0,0]).set_duration(audio_master.duration)
blackmask.set_audio(audio_master)

for i in range(2):#len(delays)):

    # LOAD VIDEO AND KILL AUDIO
    video = VideoFileClip(datapath+"40"+str(i+1)+".mp4").volumex(0) # Load and kill volume

    # VIDEO PADDING WITH DELAY VALUE
    delay = delays[i]
    video_pad = CompositeVideoClip([blackmask, video.set_start(delay)])
    
    #VIDEO AUDIO
    video_pad = video_pad.set_audio(audio_master)
    
    #VIDEO OUTPUT
    video_pad.write_videofile(filename=outputpath+"video_pad_"+str(i+1)+".mp4")#, fps="30")

In [16]:
print("Delay for this video is "+ str(delays[1]))
Video(outputpath+"video_pad_2.mp4", width=480, height=480) 

Delay for this video is 5.248


# 5. Input to system and see what we got!

Input the padded videos into [GIFU](https://gifu.eecs.qmul.ac.uk/) system the blackbox.

In [18]:
Video("_assets/stack_final.mp4", width=480, height=480) 

In [20]:
Video("_assets/Result_expected.mp4", width=480, height=480) 

#### MIT License

Copyright (c) 2019, Eungbean Lee

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.